In [1]:
import tensorflow as tf
import os
from matplotlib import pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
train_dir = 'train/'
val_dir = 'test/'
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                    rescale = 1./255,
                    rotation_range=30,
                    horizontal_flip = True,
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    fill_mode='nearest')

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                    rescale = 1./255)

In [3]:
train_data = train_datagen.flow_from_directory(
                                                train_dir,
                                                color_mode='grayscale',
                                                target_size=(48, 48))

val_data = val_datagen.flow_from_directory(
                                                val_dir,
                                                color_mode='grayscale',
                                                target_size=(48, 48))

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'neutral', 'sad', 'surprise']
img, label = train_data.__next__()
len(img)

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [26]:
plt.figure(figsize=(15, 15))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.imshow(img[i], cmap='gray')
    plt.title(class_labels[label[i].argmax()])
    plt.axis(False)
    

NameError: name 'plt' is not defined

In [27]:
img[0].shape

NameError: name 'img' is not defined

In [28]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu', input_shape=(48, 48, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(7, activation='softmax')
])



model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 46, 46, 64)        640       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 21, 21, 64)        36928     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 10, 10, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 8, 8, 64)          36928     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 4, 4, 64)        

In [30]:
history = model.fit(train_data,
                    steps_per_epoch=len(train_data),
                    epochs=5,
                    validation_data=val_data,
                    validation_steps=len(val_data))

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

## Trying Transfer Learning - Feature Extraction

In [18]:
train_data = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                         label_mode='categorical',
                                                         image_size=(224, 224))

test_data = tf.keras.utils.image_dataset_from_directory(val_dir,
                                                        label_mode='categorical',
                                                        image_size=(224, 224))

Found 28709 files belonging to 7 classes.
Found 7178 files belonging to 7 classes.


In [19]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

inputs = tf.keras.layers.Input(shape=(224, 224, 3))
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D(name='avg_pool_layer')(x)
output = tf.keras.layers.Dense(7, activation='softmax', name='output_layer')(x)

model_2 = tf.keras.Model(inputs, output)

In [20]:
model_2.compile(loss='categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])

model_2.fit(train_data,
            steps_per_epoch=len(train_data),
            epochs=5,
            validation_data=val_data,
            validation_steps=len(val_data))

Epoch 1/5
898/898 [==============================] - ETA: 0s - loss: 1.4791 - accuracy: 0.4365

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [14]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [ ]:
model_2.summary()

In [ ]:
# model.save('my_model_face_recognition_100_epochs.h5')

# PREDICTING

In [ ]:
from keras.models import load_model
my_model = load_model('my_model_face_recognition_100_epochs.h5')

In [ ]:
labels, predictions = [], []
for _ in range(200):
    test_img, labels_temp = val_generator.__next__()
    predictions_temp = my_model.predict(test_img)
    predictions_temp = np.argmax(predictions_temp, axis=1)
    labels_temp = np.argmax(labels_temp, axis=1)
    labels+=list(labels_temp)
    predictions+=list(predictions_temp)

In [ ]:
new_class_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
for i in range(len(labels)):
    labels[i] = new_class_labels[labels[i]]
    predictions[i] = new_class_labels[predictions[i]]

In [ ]:
for i in range(batch_size):
    plt.imshow(test_img[i], cmap = 'gray')
    plt.title(f'Right: {labels[i]}\n Prediction: {predictions[i]}')
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
from seaborn import heatmap

In [ ]:
fig, ax = plt.subplots(figsize=(15,13))   
heatmap(confusion_matrix(labels, predictions), annot = True, fmt='g')

# OBTAINING THE CAMERA

In [ ]:
from cv2 import VideoCapture, resize, imshow, COLOR_BGR2GRAY, cvtColor, CascadeClassifier, rectangle
from skimage import transform

In [ ]:
vid = VideoCapture(0)

# while vid.isOpened():
#     ret, frame = vid.read()
#     cv2.imshow(frame)

In [ ]:
ret, frame = vid.read()

In [ ]:
frame = cvtColor(frame, COLOR_BGR2GRAY)
plt.imshow(frame, cmap='gray')

In [ ]:
face_cascade = CascadeClassifier('haarcascade_frontalface_default.xml')
faces = face_cascade.detectMultiScale(frame, 1.1, 4)

In [ ]:
for (x, y, w, h) in faces:
    rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
# Display the output
plt.imshow(frame, cmap='gray')

In [ ]:
face = frame[y:y+h, x:x+w]
plt.imshow(face, cmap = 'gray')

In [ ]:
face = resize(face, (48, 48))
plt.imshow(face, cmap='gray')

# PREDICTING MY PHOTO

In [ ]:
from keras.models import load_model
my_model = load_model('my_model_face_recognition_100_epochs.h5')

In [ ]:
# face_r = resize(face, (48, 48))
face_r = np.expand_dims(face, axis=0)
face_r = face_r/255.0
prediction = my_model.predict(face_r)
predictions_temp = np.argmax(prediction, axis=1)
new_class_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
prediction = np.argmax(prediction)
prediction = new_class_labels[prediction]

In [ ]:
print(prediction)

In [ ]:
from sklearn.metrics import confusion_matrix
from seaborn import heatmap
import numpy as np
heatmap(confusion_matrix(np.array([1, 2, 3, 2, 3, 4, 6, 4, 3]), np.array([2, 3, 4, 2, 3, 4, 6, 4, 3])), annot=True, fmt='g')

In [ ]:
# symbol = ['Clubs', 'Diamonds', 'Hearths', 'Spades']
symbol = ['Clubs', 'Diamonds', 'Hearths', 'Spades', 'Joker']
types = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A', 'Joker']
a=[]

for i in range(len(symbol)):
    b=[]
    for g in range(len(types)):
        b.append(52)
    a.append(b)

a=np.array(a)
a[:, -1] = 0
a[-1, :] = 0
a[-1, -1] = 51
a[0, 0] = 51
a[1, 5] = 51
a[2, 1] = 51
a[2, -2] = 51
a[-2, 0] = 51
a[-2, 2:3] = 51
a[0, 2] = 62
# a[-1, 6] = 51

# heatmap(confusion_matrix(np.array(b), np.array(a)), annot=True, fmt='g')
heatmap(a,cmap="flare", annot=True, linewidths=0.1, linecolor="gray", xticklabels=types, yticklabels=symbol, vmin=1, vmax=62)